In [6]:
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup as bs
from pandas.io.json import json_normalize

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

## Import Data